In [1]:
! uv pip install --pre anndata scanpy "dask>=2022.09.2,<2024.8.0" "dask-ml>=2022.09.2,<2024.8.0" "dask[distributed]>=2022.09.2,<2024.8.0" "bokeh!=3.0.*,>=2.4.0"

Audited 6 packages in 2.32s


In [2]:
from dask.distributed import Client, LocalCluster
import dask.array as da
import dask
from scipy import sparse
import numpy as np
import anndata as ad
import scanpy as sc
from pathlib import Path
import h5py

# Setup

To begin, let's set up some compute resources locally.  We'll use `LocalCluster` because this tutorial will run on a laptop, but `dask` and `dask-cuda` offer deep customization + heterogeneous compute support:

- https://docs.dask.org/en/stable/deploying.html
- https://docs.rapids.ai/api/dask-cuda/nightly/

We'll then navigate to the URL listed to be able to monitor our resources - this handy functionality comes for free with `dask`: http://127.0.0.1:8787/status

*Note* We will only be working with numeric array data.  Both [dask.DataFrame](https://docs.dask.org/en/stable/dataframe.html) and [xarray](https://docs.xarray.dev/en/stable/) offer stable dataframe functionality.  `AnnData` objects will hopefully soon be integrated with `xarray`.

In [3]:
cluster = LocalCluster(n_workers=25)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 25
Total threads: 225,Total memory: 1.95 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8209,Workers: 25
Dashboard: http://127.0.0.1:8787/status,Total threads: 225
Started: Just now,Total memory: 1.95 TiB
Comm: tcp://127.0.0.1:8143,Total threads: 9
Dashboard: http://127.0.0.1:8223/status,Memory: 80.00 GiB
Nanny: tcp://127.0.0.1:8373,


# First steps

Let's get a feel for `dask.array.Array` by using `map_blocks` to generate some data.  Some notes:

- [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html) support with `dask` is crucial for our purposes, but somewhat lacking.  We are working on this!  But expect some sharp edges here!
- Note `chunks`; we have to be very specific about this information because `dask` uses it to compute `.shape` - there is no `shape` here!

# A real computation

Let's try to do something, calculate a gram/covaraince matrix using normal dense data. 

$$\Large X^T X$$

**Note** Take a look at the chunking. Why does this make sense for a lot of single-cell analysis?  Especially that in `scanpy`?

In [4]:
M, N = 2_000_000, 10_000
arr_dask = da.random.random((M, N), chunks=(1_000, N))
arr_dask

dask.array<random_sample, shape=(2000000, 10000), dtype=float64, chunksize=(1000, 10000), chunktype=numpy.ndarray>

In [5]:
gram_matrix = arr_dask.T @ arr_dask
gram_matrix

dask.array<_chunk_sum-aggregate, shape=(10000, 10000), dtype=float64, chunksize=(10000, 10000), chunktype=numpy.ndarray>

In [6]:
%time gram_matrix.compute()

CPU times: user 36.6 s, sys: 15.6 s, total: 52.2 s
Wall time: 1min 50s


array([[666866.84569336, 500061.38819278, 499950.09634675, ...,
        499761.94383004, 499948.85923108, 499995.48534689],
       [500061.38819278, 666693.97790411, 499926.66760091, ...,
        499943.28579016, 500034.52630128, 499858.73719638],
       [499950.09634675, 499926.66760091, 666458.5009242 , ...,
        499669.74021847, 499633.75845863, 499689.66072292],
       ...,
       [499761.94383004, 499943.28579016, 499669.74021847, ...,
        666337.31245501, 499940.22253482, 499739.58033563],
       [499948.85923108, 500034.52630128, 499633.75845863, ...,
        499940.22253482, 666508.82048036, 499574.28689934],
       [499995.48534689, 499858.73719638, 499689.66072292, ...,
        499739.58033563, 499574.28689934, 666392.69851526]])

We won't do the same thing in `numpy` because it takes much longer i.e., generating the matrix + doing the computation.  In fact, the generation in `numpy` takes about as long as the `dask` generation + computation combined!

In [7]:
%%time
# arr_np = np.random.random((M, N))

CPU times: user 1min 43s, sys: 22.3 s, total: 2min 6s
Wall time: 1min 50s


In [8]:
%%time 
# arr_np.T @ arr_np

CPU times: user 1h 4min 10s, sys: 13min 45s, total: 1h 17min 56s
Wall time: 1min 21s


array([[666242.22047548, 499712.24062519, 499956.75107923, ...,
        499483.12112549, 499794.57351165, 499806.09208033],
       [499712.24062519, 666543.26948798, 499951.19647375, ...,
        499430.20490301, 500113.11793398, 499929.0571049 ],
       [499956.75107923, 499951.19647375, 667156.77081061, ...,
        499829.13176151, 500309.92173935, 500270.21216223],
       ...,
       [499483.12112549, 499430.20490301, 499829.13176151, ...,
        665944.57011269, 499450.67561122, 499633.12987764],
       [499794.57351165, 500113.11793398, 500309.92173935, ...,
        499450.67561122, 666766.70365393, 499906.67535439],
       [499806.09208033, 499929.0571049 , 500270.21216223, ...,
        499633.12987764, 499906.67535439, 666621.6326087 ]])

## Sparse

Now let's try to do the same thing in sparse.  We'll create our matrix chunk-by-chunk.

In [7]:
n_blocks = 1000
M_block = M / n_blocks
def make_chunk():
    return sparse.random(M_block, N, format="csr")

arr = da.map_blocks(
    make_chunk,
    meta=sparse.random(2, 2, format="csr"),
    dtype=np.float64,
    chunks=((M_block,) * n_blocks, (N,) * 1)
)
arr

dask.array<make_chunk, shape=(2000000, 10000), dtype=float64, chunksize=(2000, 10000), chunktype=scipy.csr_matrix>

In [10]:
# (arr.T @ arr).compute()

2024-09-12 11:19:37,444 - distributed.worker - WARNING - Compute Failed
Key:       ('chunk_sum-f0d8e4e2bf1f322cf55f1e6915e55c5a', 0, 697, 0)
State:     executing
Function:  subgraph_callable-9519ac31b06da7b8690e005fa3efc687
args:      (<2000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 200000 stored elements in Compressed Sparse Row format>, <2000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 200000 stored elements in Compressed Sparse Row format>)
kwargs:    {}
Exception: "ValueError('matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)')"
Traceback: '  File "/ictstr01/home/icb/ilan.gold/venv/lib/python3.10/site-packages/dask/optimization.py", line 1001, in __call__\n    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))\n  File "/ictstr01/home/icb/ilan.gold/venv/lib/python3.10/site-packages/dask/core.py", line 157, in get\n    result = _execute_task(task, cache)\n  

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

But that doesn't quite work....support for `scipy.sparse` in `dask` is not great (but we are working on it! upstream!)

As I said, sharp edges.  So let's think about how to do this with a bit of custom code.  This is not a bad exercise if you ever want to do anything custom with your `AnnData.X` anyway!  We can think of our computation as follows.

First define the block size

$$ b = floor(\frac{M}{n_{blocks}}) $$

The our sub-matrix structure, defined by "chunks" of our matrix (as we declared in `dask`)

$$ \mathbf{\hat{X}}_j = [\mathbf{X}_{j \cdot  b}, \ldots, \mathbf{X}_{(j + 1) \cdot b)}] \in \mathbb{R}^{b \times N} $$

And finally the result we want, the covariance matrix:

$$\Large \mathbf{X}^T \mathbf{X} = \sum_{j = 0}^{n_{blocks}} \mathbf{\hat{X}}_j^T \mathbf{\hat{X}}_j \in \mathbb{R}^{M \times N}$$

We'll also compute `log1p` before hand.  This will demonstrate the **lazy** aspect of `dask` even more.  We now have three steps - an elementwise operation, the mapping to covariance blocks, and finally the reduction.

In [8]:
def __gram_block(block):
    return (block.T @ block).toarray()[None, ...]

lop1p_matrix = arr.map_blocks(
    lambda x: x.log1p(),
    meta=sparse.random(2, 2, format="csr")
)

gram_matrix = lop1p_matrix.map_blocks(
    __gram_block,
    new_axis=(1,),
    chunks=((1,) * n_blocks, (arr.shape[1],), (arr.shape[1],)),
    meta=np.array([]),
    dtype=arr.dtype,
).sum(axis=0)

Note some things - we convert to dense to save on memory since the block covariance matrix is fairly dense anyway.  We also have to be explicit with `dask` about our `chunks` structure - we add a different axis so that we can sum sum over it, and we must be explicit about this.  Now let's compute our result!  Note that we have done **no** computation at this point - everything is lazy!

In [9]:
%%time
computed_gram_matrix = gram_matrix.compute()
computed_gram_matrix.shape

CPU times: user 12.6 s, sys: 6.58 s, total: 19.2 s
Wall time: 1min 27s


(10000, 10000)

# Working with `AnnData`

Let's now do something with the `scverse` ecosystem.  We'll begin by reading in the data.  Note the use of our new APIs, [`anndata.read_elem`](https://anndata.readthedocs.io/en/latest/generated/anndata.read_elem.html) and [`anndata.experimental.read_elem_as_dask`](https://anndata.readthedocs.io/en/latest/generated/anndata.experimental.read_elem_as_dask.html).

In [10]:
file_name = "cell_atlas.h5ad"
if not Path(file_name).exists():
    !wget https://datasets.cellxgene.cziscience.com/82eac9c1-485f-4e21-ab21-8510823d4f6e.h5ad -O file_name

In [11]:
with h5py.File(file_name, "r") as f:
    adata = ad.AnnData(
        X = ad.experimental.read_elem_as_dask(f["X"]),
        obs = ad.read_elem(f["obs"]),
        var = ad.read_elem(f["var"])
    )
adata

AnnData object with n_obs × n_vars = 1462702 × 27714
    obs: 'celltype', 'majorType', 'City', 'sampleID', 'donor_id', 'Sample type', 'CoVID-19 severity', 'Sample time', 'Sampling day (Days after symptom onset)', 'BCR single cell sequencing', 'TCR single cell sequencing', 'Outcome', 'Comorbidities', 'COVID-19-related medication and anti-microbials', 'Leukocytes [G over L]', 'Neutrophils [G over L]', 'Lymphocytes [G over L]', 'Unpublished', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'suspension_type', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length'

In [12]:
adata.X

dask.array<make_dask_chunk, shape=(1462702, 27714), dtype=float32, chunksize=(1000, 27714), chunktype=scipy.csr_matrix>

## What can we do?

We'll step throught the major steps we have `dask`-ified in `scanpy`.  This is a somehwat normal workflow - filtering, variable genes, and PCA.  More to come soon!

In [13]:
%%time
adata.layers["counts"] = adata.X.copy()  # Making sure we keep access to the raw counts
sc.pp.normalize_total(adata, target_sum=1e4)

CPU times: user 18.7 ms, sys: 6.34 ms, total: 25 ms
Wall time: 24.7 ms


In [14]:
%%time
sc.pp.log1p(adata)

CPU times: user 3.29 ms, sys: 0 ns, total: 3.29 ms
Wall time: 3.3 ms


In [15]:
%%time
sc.pp.highly_variable_genes(adata)

CPU times: user 39.9 s, sys: 1.79 s, total: 41.7 s
Wall time: 48 s


In [16]:
adata.var['highly_variable'].sum()

2105

In [17]:
adata.layers["dense"] = adata.X.rechunk((10_000, -1)).map_blocks(
    lambda x: x.toarray(), dtype=adata.X.dtype, meta=np.array([])
)

In [18]:
%%time
sc.pp.pca(adata, layer="dense")

CPU times: user 16.6 s, sys: 2.32 s, total: 18.9 s
Wall time: 39.6 s


In [19]:
%%time
adata.obsm["X_pca"] = adata.obsm["X_pca"].compute()

2024-09-12 12:15:27,935 - distributed.utils_perf - WARNING - full garbage collections took 42% CPU time recently (threshold: 10%)
2024-09-12 12:15:28,171 - distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)


CPU times: user 13.6 s, sys: 1.61 s, total: 15.2 s
Wall time: 16.9 s


## GPU anyone?

What if we want to have some real fun?  What about GPU?

In [20]:
! uv pip install dask-cuda cupy-cuda12x rmm-cu12

Audited 3 packages in 828ms


In [21]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cupy as cp
import rmm
import numpy as np
from rmm.allocators.cupy import rmm_cupy_allocator
from cupyx.scipy import sparse as cp_sparse
import anndata as ad
from scipy import sparse
import h5py

# Set up rapids memory management, which allows spilling to disk among other nice features
def set_mem():
    rmm.reinitialize(managed_memory=True)
    cp.cuda.set_allocator(rmm_cupy_allocator)
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="0,1,2,3,4,5,6,7,8")
client = Client(cluster)
client.run(set_mem)
client

/ictstr01/home/icb/ilan.gold/scverse_tutorial/venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 8105 instead
  warnings.warn(
/ictstr01/home/icb/ilan.gold/scverse_tutorial/venv/lib/python3.10/site-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 8, setting default affinity
  warnings.warn(
2024-09-12 12:16:34,518 - distributed.nanny - ERROR - Failed to initialize worker
Traceback (most recent call last):
  File "/ictstr01/home/icb/ilan.gold/scverse_tutorial/venv/lib/python3.10/site-packages/distributed/nanny.py", line 952, in run
    worker = worker_factory()
  File "/ictstr01/home/icb/ilan.gold/scverse_tutorial/venv/lib/python3.10/site-packages/distributed/worker.py", line 715, in __init__
    ServerNode.__init__(
  File "/ictstr01/home/icb/ilan.gold/scverse_tutorial/venv/lib/python3.10/site-packages/distributed/core.

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8105/status,
Dashboard: http://127.0.0.1:8105/status,Workers: 8
Total threads: 8,Total memory: 1.74 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8409,Workers: 8
Dashboard: http://127.0.0.1:8105/status,Total threads: 8
Started: 1 minute ago,Total memory: 1.74 TiB
Comm: tcp://127.0.0.1:8229,Total threads: 1
Dashboard: http://127.0.0.1:8197/status,Memory: 222.22 GiB
Nanny: tcp://127.0.0.1:8089,


In [22]:
with h5py.File("cell_atlas.h5ad", "r") as f:
    adata = ad.AnnData(
        X = ad.experimental.read_elem_as_dask(f["X"]),
        obs = ad.read_elem(f["obs"]),
        var = ad.read_elem(f["var"])
    )
adata.X = adata.X.map_blocks(
    cp_sparse.csr_matrix,
    meta=cp_sparse.csr_matrix(sparse.random(1, 1, format="csr"), dtype=adata.X.dtype),
    dtype=adata.X.dtype
)
adata.X

dask.array<csr_matrix, shape=(1462702, 27714), dtype=float32, chunksize=(1000, 27714), chunktype=cupyx.csr_matrix>

Let's do the same thing, but now with GPU functionality.  Note the different `meta` than before, but otherwise, remarkably similar!

In [23]:
def __gram_block_densify(block):
    densified = block.toarray()
    return (densified.T @ densified)[None, ...]

In [24]:
def __gram_block(block):
    return (block.T @ block).toarray()[None, ...]

In [25]:
n_blocks = len(adata.X.to_delayed().ravel())
gram_matrix_gpu_lazy = adata.X.map_blocks(
    __gram_block,
    new_axis=(1,),
    chunks=((1,) * n_blocks, (adata.shape[1],), (adata.shape[1],)),
    meta=cp.array([], dtype=adata.X.dtype),
    dtype=adata.X.dtype,
).sum(axis=0)
gram_matrix_gpu_lazy

dask.array<sum-aggregate, shape=(27714, 27714), dtype=float32, chunksize=(27714, 27714), chunktype=cupy.ndarray>

In [20]:
# gram_matrix_gpu_lazy.compute()

2024-09-12 11:34:05,032 - distributed.worker - WARNING - Compute Failed
Key:       ('sum-263b98f08180ed267b5ccab308690283', 1022, 0, 0)
State:     executing
Function:  execute_task
args:      ((subgraph_callable-41356ea3a7459b12cbc3205022c1322d, functools.partial(<function make_dask_chunk at 0x7fdb09149f30>, PosixPath('cell_atlas.h5ad'), '/X', wrap=<function sparse_dataset at 0x7fdb090ce7a0>), (<class 'tuple'>, ['block_info']), {None: {'shape': (1462702, 27714), 'num-chunks': (1463, 1), 'array-location': [(1022000, 1023000), (0, 27714)], 'chunk-location': (1022, 0), 'chunk-shape': (1000, 27714), 'dtype': dtype('float32')}}))
kwargs:    {}
Exception: "CuSparseError('CUSPARSE_STATUS_INSUFFICIENT_RESOURCES: insufficient resources')"
Traceback: '  File "/ictstr01/home/icb/ilan.gold/venv/lib/python3.10/site-packages/dask/optimization.py", line 1001, in __call__\n    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))\n  File "/ictstr01/home/icb/ilan.gold/venv/lib/python3.10

CuSparseError: CUSPARSE_STATUS_INSUFFICIENT_RESOURCES: insufficient resources

In [21]:
gram_matrix_gpu_lazy = adata.X.map_blocks(
    __gram_block_densify,
    new_axis=(1,),
    chunks=((1,) * n_blocks, (adata.shape[1],), (adata.shape[1],)),
    meta=cp.array([], dtype=adata.X.dtype),
    dtype=adata.X.dtype,
).sum(axis=0)

In [22]:
%%time
# gram_matrix_gpu_in_memory = gram_matrix_gpu_lazy.compute()
# gram_matrix_gpu_in_memory

CPU times: user 1min 17s, sys: 25.6 s, total: 1min 42s
Wall time: 8min 53s


array([[6.6108523e+04, 2.6539505e-01, 2.6594014e+03, ..., 1.1166708e+01,
        1.7306891e-01, 7.7735577e+02],
       [2.6539505e-01, 3.1194468e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.6594014e+03, 0.0000000e+00, 3.4531633e+04, ..., 1.2455404e+01,
        0.0000000e+00, 3.9746011e+02],
       ...,
       [1.1166708e+01, 0.0000000e+00, 1.2455404e+01, ..., 2.3650439e+02,
        0.0000000e+00, 1.5137998e+00],
       [1.7306891e-01, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        3.4908867e+00, 0.0000000e+00],
       [7.7735577e+02, 0.0000000e+00, 3.9746011e+02, ..., 1.5137998e+00,
        0.0000000e+00, 7.6181641e+03]], dtype=float32)

Hmmmm, we had to densify everything, which really slows things down....maybe that's one reason why [`rapids_singlecell`](https://rapids-singlecell.readthedocs.io/) exists? And maybe why we're working on this problem specifically?

In [26]:
! uv pip install git+https://github.com/scverse/rapids_singlecell@dask_mg_support[rapids12] --extra-index-url https://pypi.nvidia.com

Resolved 100 packages in 8.65s                                               
Audited 100 packages in 0.44ms


In [27]:
import rapids_singlecell as rsc

/ictstr01/home/icb/ilan.gold/scverse_tutorial/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
adata.X = adata.X.persist()

In [29]:
%%time
rsc.pp.highly_variable_genes(adata)
rsc.pp.pca(adata)

CPU times: user 22.1 s, sys: 2.77 s, total: 24.8 s
Wall time: 43.1 s


In [30]:
! uv pip freeze

aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
anndata==0.11.0rc1
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-api-compat==1.8
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
async-timeout==4.0.3
attrs==24.2.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.1.0
bokeh==3.6.0.dev1
cachetools==5.5.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.3.2
click==8.1.7
cloudpickle==3.0.0
comm==0.2.2
contourpy==1.3.0
cuda-python==12.6.0
cudf-cu12==24.8.2
cugraph-cu12==24.8.0
cuml-cu12==24.8.0
cupy-cuda12x==13.3.0
cycler==0.12.1
dask==2024.7.1
dask-cuda==24.8.2
dask-cudf-cu12==24.8.2
dask-expr==1.1.9
dask-glm==0.3.2
dask-ml==2024.4.4
debugpy==1.8.5
decorator==5.1.1
defusedxml==0.7.1
distributed==2024.7.1
distributed-ucxx-cu12==0.39.1
exceptiongroup==1.2.2
executing==2.1.0
fastjsonschema==2.20.0
fastrlock==0.8.2
fonttools==4.53.1
fqdn==1.5.1
frozenlist==1.4.1
fsspec==2024.9.0
h11==0.14.0
h5py==3.11.0
httpcore==1.0.5
httpx==0.27.2
idna==3.8
imageio==2.35.1
impor